This research is about a Otsu method-based moving object detection method

## Table of contents
- [Project structure](#anchor1)
- [Foreground detection](#anchor2)
  - [How to use](#anchor3)
  - [Use cases](#anchor4)
- [Background detection](#anchor5)
  - [What is ANN](#anchor6)
  - [How to use](#anchor7)
  - [Use cases](#anchor8)
- [paper link](paper/A_Moving_Object_Detection_Method_Based_on_the_Otsu_Method.pdf)

### Project structure <a name="anchor1"></a>
There are four folders: 
- test_img
- foreground_img
- binarization_img
- paper

*'test_img'* contains 11 test images in which *'000000.jpg'* is the background image; 

*'foreground_img'* is for saving images with the foreground; 

*'binarization_img'* contains the binarized images; 

Related thesis is placed in the *'paper'* folder. 

### Foreground detection <a name="anchor2"></a>
#### How to use <a name="anchor3"></a>
Put your images in a folder, and use 
```
get_foreground_median(folder_path, output_path, binarization='n')
```
function, in which `folder_path` is images path, `output_path` to put output images. 

And you can `binarization` images with the Otsu method. 

In [7]:
from os import listdir
from PIL import Image
import numpy as np
import cv2

def get_foreground_median(folder_path, output_path, binarization='n'):
    
    ###################
    # read image name #
    ###################
    img_name_list = []
    for img_name in listdir(folder_path):
        img_name_list.append(str(img_name))
#     print(img_name_list)
    
    ##############
    # read image #
    ##############
    img_list = []
    for img_name in img_name_list:
        img_list.append(np.asarray(Image.open(folder_path + img_name).convert('L'))) # convert to gray-scale
#     print(img_list[0].shape)

    ###########################
    # calculate median image #
    ###########################
    img_array = np.asarray(img_list, np.float64) # convert list to array
#     print(img_array.shape) # (11, 480, 1140)
    average_img = np.median(img_array, axis=0)
#     print(average_img.shape) #  (480, 1140)
#     print(average_img)
    
    ###########################################
    # minus average image from original image #
    ###########################################
    img_array -= average_img
#     print(img_array)
    img_array = np.abs(img_array)
#     print(img_array)
    img_array = np.around(img_array).astype(np.uint8)
#     print(img_array)

    ################
    # binarization #
    ################
    if binarization == 'y':
        for i in range(len(img_array)):
            _, img_array[i] = cv2.threshold(img_array[i], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    ##############
    # output img #
    ##############
    for i in range(len(img_array)):
        Image.fromarray(img_array[i]).save(output_path + img_name_list[i], compress_level=0)

#### Use cases <a name="anchor4"></a>

In [8]:
test_img_path = 'test_img/'
foreground_img_path = 'foreground_img/'
binarization_img_path = 'binarization_img/'

##################################
# get images with the foreground #
##################################
get_foreground_median(test_img_path, foreground_img_path)

############################
# get the binarized images #
############################
get_foreground_median(test_img_path, binarization_img_path, binarization='y')

### Background detection <a name="anchor5"></a>
<img src=binarization_img/000001.jpg width=50% />

The image above shows an good binarizatione xample. However, to the background image *'000000,jpg'*, some pixels are regarded as foreground as there is no foreground, like the image below: 

<img src=binarization_img/000000.jpg width=50% />

We use **Average Nearest Neighbor (ANN)** to distinguish these background only images. 

#### What is ANN <a name="anchor6"></a>
The **ANN** is calculated as the **observed average distance** divided by the **expected average distance**, and can be expressed as
$$
ANN=\frac{\overline{D_o}}{\overline{D_e}},
$$
in which $\overline{D_o}$ is the **observed average distance** and $\overline{D_e}$ is the **expected average distance**, and they can be expressed as
$$
\overline{D_o}=\frac{\sum_{i=1}^{n}d_i}{n}
$$

$$
\overline{D_e}=\frac{0.5}{\sqrt{\frac{n}{A}}}
$$
in which $d_i$ is the distance between a white pixel to its nearest other white pixel, and $n$ is the indexes of the white pixels, $A$ is the area of an image. 

#### How to use <a name="anchor7"></a>
Use 
```
cal_ANN(img_path)
```
function to calculate ANN to distinguish background images and foreground images. 

In [56]:
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from math import sqrt

palette = [0, 0, 0, 255, 255, 255]

def cal_ANN(img_path):
    
    ##############
    # read image #
    ##############
    img = Image.open(img_path).convert('L') # convert to gray-scale
    img = np.asarray(img) # convert to array
#     print(img.shape)

    ################
    # binarization #
    ################
    _, img = cv2.threshold(img, 127, 1, cv2.THRESH_BINARY) # let white pixel value 1
#     print(img)
#     img = Image.fromarray(img).convert('P')
#     img.putpalette(palette)
#     plt.figure()
#     plt.imshow(img)

    #################
    # calculate ANN #
    #################
    ANN = _cal_ANN(img)
    return ANN
    
def _cal_ANN(img):
    
    #############################
    # get foreground coordinate #
    #############################
    foreground_coordinate = np.argwhere(img != 0)
#     print(foreground_coordinate)
    n = len(foreground_coordinate)
#     print(n)

    tree = KDTree(foreground_coordinate)
    
    #############################################################################
    # calculate diatance to the second closest point (the closest is to itself) #
    #############################################################################
    distance, _ = tree.query(foreground_coordinate, k=[2])
#     print(distance)

    ################
    # sum distance #
    ################
    distance = np.sum(distance)
#     print(distance)
    
    ######
    # Do #
    ######
    Do = distance / n
#     print(Do)

    Y = img.shape[0]
    X = img.shape[1]
    A = Y * X
#     print(A)
    
    ######
    # De #
    ######
    De = 0.5 / sqrt(n / A)
    
    return Do / De

#### <font color=DCDCDC>Code test</font> <a name="anchor8"></a>

In [57]:
test_img = np.array([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0]).reshape((3, 4))
print(test_img)
ANN = _cal_ANN(test_img)
print(ANN)

[[0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]]
1.4142135623730951


#### Use cases <a name="anchor8"></a>

In [58]:
######################
# background img ANN #
######################
test_img_path = 'binarization_img/000000.jpg'
ANN = cal_ANN(test_img_path)
print(ANN)

#############################
# having foreground img ANN #
#############################
test_img_path = 'binarization_img/000001.jpg'
ANN = cal_ANN(test_img_path)
print(ANN)

1.087569832557552
0.14013132150041546
